In [ ]:
import pandas as pd

In [ ]:
# Lire le CSV result
result = pd.read_csv('data_quantitative.csv')

# Lire le CSV data_LSA_archive
data_LSA = pd.read_csv('data_LSA_archive.csv')

# Convertir la colonne Publication_Date en format datetime
data_LSA['Publication_Date'] = pd.to_datetime(data_LSA['Publication_Date'])

# Extraire le mois et l'année de Publication_Date pour avoir une colonne 'Month_Year'
data_LSA['Month_Year'] = data_LSA['Publication_Date'].dt.to_period('M')

# Compter le nombre d'articles par mois
articles_per_month = data_LSA.groupby('Month_Year').size()

# Calculer le nombre total d'articles dans data_LSA
total_articles = len(data_LSA)

# Ajouter une colonne "LSA_Articles_Ratio" dans result
# Nous allons faire un merge pour joindre les données d'articles LSA par mois
result['Month_Year'] = pd.to_datetime(result['Month_Year'], format='%Y-%m')
result['Month_Year'] = result['Month_Year'].dt.to_period('M')

# Fusionner avec les articles LSA par mois
result = result.merge(articles_per_month, on='Month_Year', how='left')

# Calculer le ratio du nombre d'articles LSA par rapport au nombre total d'articles
result['LSA_Articles_Ratio'] = result['Month_Year'].map(lambda x: articles_per_month.get(x, 0) / total_articles)

# Sauvegarder le DataFrame avec la nouvelle colonne dans un nouveau CSV
result.to_csv('data_quantitative.csv', index=False)

print("Le fichier CSV avec le ratio des articles LSA par mois a été créé avec succès.")


In [ ]:
# Lire le CSV data_leMonde
data_leMonde = pd.read_csv('data_leMonde.csv')

# Convertir la colonne Publication_Date en format datetime
data_leMonde['Publication_Date'] = pd.to_datetime(data_leMonde['Publication_Date'])

# Extraire le mois et l'année de Publication_Date pour avoir une colonne 'Month_Year'
data_leMonde['Month_Year'] = data_leMonde['Publication_Date'].dt.to_period('M')

# Compter le nombre d'articles par mois
articles_per_month_leMonde = data_leMonde.groupby('Month_Year').size()

# Calculer le nombre total d'articles dans data_leMonde
total_articles_leMonde = len(data_leMonde)

# Ajouter une colonne "LeMonde_Articles_Ratio" dans result
# Nous allons faire un merge pour joindre les données d'articles LeMonde par mois
result['Month_Year'] = pd.to_datetime(result['Month_Year'], format='%Y-%m')
result['Month_Year'] = result['Month_Year'].dt.to_period('M')

# Fusionner avec les articles LeMonde par mois
result = result.merge(articles_per_month_leMonde, on='Month_Year', how='left')

# Calculer le ratio du nombre d'articles LeMonde par rapport au nombre total d'articles
result['LeMonde_Articles_Ratio'] = result['Month_Year'].map(lambda x: articles_per_month_leMonde.get(x, 0) / total_articles_leMonde)

In [ ]:
# Calculer la moyenne des trois colonnes
result['Average'] = result[['normalized_min_max', 'LSA_Articles_Ratio', 'LeMonde_Articles_Ratio']].mean(axis=1)

# Créer un nouveau DataFrame avec uniquement 'Month_Year' et la moyenne calculée
final_result = result[['Month_Year', 'Average']]

# Sauvegarder le nouveau DataFrame dans un CSV
final_result.to_csv('data_quantitative.csv', index=False)

In [ ]:
# Lire les CSV
data_LSA = pd.read_csv('data_LSA_archive.csv')
data_LeMonde = pd.read_csv('data_LeMonde.csv')
data_huffpost = pd.read_csv('data_huffpost.csv')

# Convertir les dates dans un format commun (Mois-Annee)
# Pour data_LSA, la colonne 'Publication_Date' est au format 'YYYY-MM-DD HH:MM:SS'
data_LSA['Publication_Date'] = pd.to_datetime(data_LSA['Publication_Date'])
data_LSA['Month_Year'] = data_LSA['Publication_Date'].dt.to_period('M')

# Pour data_LeMonde, la colonne 'Date' est au format 'DD-MM-YYYY'
data_LeMonde['Date'] = pd.to_datetime(data_LeMonde['Date'], format='%d-%m-%Y')
data_LeMonde['Month_Year'] = data_LeMonde['Date'].dt.to_period('M')

# Pour data_huffpost, la colonne 'Date' est au format 'DD/MM/YYYY HH:MM'
data_huffpost['Date'] = pd.to_datetime(data_huffpost['Date'], format='%d/%m/%Y %H:%M')
data_huffpost['Month_Year'] = data_huffpost['Date'].dt.to_period('M')

# Calculer la moyenne des Sentiment_Score pour chaque mois dans chaque fichier
# Pour data_LSA
lsa_avg_sentiment = data_LSA.groupby('Month_Year')['Sentiment_Score'].mean().reset_index()

# Pour data_LeMonde
lemonde_avg_sentiment = data_LeMonde.groupby('Month_Year')['Average_Sentiment_Score'].mean().reset_index()

# Pour data_huffpost
huffpost_avg_sentiment = data_huffpost.groupby('Month_Year')['Average_Sentiment_Score'].mean().reset_index()

# Fusionner les résultats
# Fusionner les 3 DataFrames sur la colonne 'Month_Year'
merged = pd.merge(lsa_avg_sentiment, lemonde_avg_sentiment, on='Month_Year', how='outer', suffixes=('_LSA', '_LeMonde'))
merged = pd.merge(merged, huffpost_avg_sentiment, on='Month_Year', how='outer')
merged.rename(columns={'Average_Sentiment_Score': 'Average_Sentiment_Score_Huffpost'}, inplace=True)

# Calculer la moyenne des 3 moyennes de sentiment par mois
merged['Average_Sentiment_Score'] = merged[['Sentiment_Score_LSA', 'Average_Sentiment_Score_LeMonde', 'Average_Sentiment_Score_Huffpost']].mean(axis=1)

# Créer le fichier CSV final
final_result = merged[['Month_Year', 'Average_Sentiment_Score']]

# Sauvegarder le CSV final
final_result.to_csv('final_sentiment_score_by_month.csv', index=False)

print("Le fichier final_sentiment_score_by_month.csv a été créé avec succès.")
